In [1]:
from importlib.metadata import version

print(f"sec_downloader version: {version('sec_downloader')}")
print(f"sec_parser version: {version('sec_parser')}")

sec_downloader version: 0.11.1
sec_parser version: 0.56.0


# How do I parse other form types with `sec-parser`, such as 10-K, 8-K, S-1, etc.?

As of April 2023, `sec-parser` supports parsing various SEC form types beyond just 10-Q, such as 10-K, 8-K, S-1, and more. Basically, it supports parsing any structured text forms.

However, currently only parsing 10-Q top level section types is supported.

This gives us two methods to parse other form types with `sec-parser`:

1. Ignore the warnings from the parsing step that identifies the 10-Q top level section types
2. Skip the parsing step that identifies the 10-Q top level section types


## Method 1: Ignore warnings from the parsing step that identifies the 10Q top level section types

In [10]:
import warnings
from sec_downloader import Downloader
import sec_parser as sp

dl = Downloader("MyCompanyName", "email@example.com")
html = dl.get_filing_html(ticker="AAPL", form="10-K")
parser = sp.Edgar10QParser()

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", message="Invalid section type for")
    elements: list = parser.parse(html)
    
tree: sp.SemanticTree = sp.TreeBuilder().build(elements)

demo_output: str = sp.render(tree)
print('\n'.join(demo_output.split('\n')[:5]))

TopSectionTitle: PART I
├── TopSectionTitle: Item 1.    Business
│   ├── TitleElement: Company Background
│   │   └── TextElement: The Company designs, manufactur...the last Saturday of September.
│   ├── TitleElement: Products


## Method 2: Skip the parsing steps related to the 10Q top level section types

In [6]:
from sec_downloader import Downloader
import sec_parser as sp
from sec_parser.processing_steps import TopSectionManagerFor10Q, IndividualSemanticElementExtractor, TopSectionTitleCheck

dl = Downloader("MyCompanyName", "email@example.com")
html = dl.get_filing_html(ticker="AAPL", form="10-K")

def without_10q_related_steps():
    all_steps = sp.Edgar10QParser().get_default_steps()
    
    # Change 1: Remove the TopSectionManagerFor10Q
    steps_without_top_section_manager = [step for step in all_steps if not isinstance(step, TopSectionManagerFor10Q)]
    
    # Change 2: Replace the IndividualSemanticElementExtractor with a new one that has the top section checks removed
    def get_checks_without_top_section_title_check():
        all_checks = sp.Edgar10QParser().get_default_single_element_checks()
        return [check for check in all_checks if not isinstance(check, TopSectionTitleCheck)]
    return [
        IndividualSemanticElementExtractor(get_checks=get_checks_without_top_section_title_check) 
        if isinstance(step, IndividualSemanticElementExtractor) 
        else step
        for step in steps_without_top_section_manager
    ]

parser = sp.Edgar10QParser(get_steps=without_10q_related_steps)
elements: list = parser.parse(html)
tree: sp.SemanticTree = sp.TreeBuilder().build(elements)

demo_output: str = sp.render(tree)
print('\n'.join(demo_output.split('\n')[90:105]))

│   │   └── TitleElement: Health and Safety
│   │       └── TextElement: The Company is committed to pro...rough its global security team.
│   ├── TitleElement: Available Information
│   │   └── TextElement: The Company’s Annual Reports on...active textual references only.
│   ├── TitleElement: Item 1A.    Risk Factors
│   │   ├── TextElement: The Company’s business, reputat...ins forward-looking statements.
│   │   └── SupplementaryText: This section should be read in ...entary Data” of this Form 10-K.
│   ├── TitleElement: Macroeconomic and Industry Risks
│   │   ├── SupplementaryText: The Company’s operations and pe...ations and financial condition.
│   │   ├── TextElement: The Company has international o...cial condition and stock price.
│   │   ├── SupplementaryText: The Company’s business can be i...d other business interruptions.
│   │   ├── TextElement: Political events, trade and oth...over all losses that may arise.
│   │   ├── SupplementaryText: Global markets for the Compan

# Your Feedback Matters to Us
We hope you found this notebook helpful! Your feedback is always welcome - feel free to open an issue or PR on our GitHub repo:
https://github.com/alphanome-ai/sec-parser
